In [10]:
from timeit import default_timer
from tkinter import *
import numpy as np
from time import time
import sys

In [11]:
text_train="The goal of this program is to identify a user from the way he or she types."
text_test ="Let us see if I can tell who you are based on your unique way of typing."

def get_user_data(text):
    """GUI that makes user type a text and outputs a dict 
    with TWOGRAMS as keys and typing times as values.
    """
    
    # Create the root window
    root = Tk()
    root.geometry('1500x200+100+100')
    root.title('Typing Recognition Program')
    
    # Create a keystroke handler
    keys=[]
    times=[]
    def key(event):
        if (event.keysym == 'Return'):
            root.destroy()
        else:
            keystroke, t = event.char, time()
            #print('(keystroke, time):', (repr(event.char),time()))
            keys.append(keystroke)
            times.append(t)

    # Create a label with instructions
    text= '\nType the following text, then press "Enter" to exit:\n\n\n'+'\"'+text+'\"\n'
    label = Label(root, width=400, height=300, text=text, justify=LEFT)
    label.pack(fill=BOTH, expand=1)
    label.bind('<Key>', key)
    label.focus_set()

    # Hand over to the Tkinter event loop
    root.mainloop()
    
    #create arrays with kestroke and time info
    keys=np.array(keys)
    times = np.array(times)-times[0]
    
    #test if the text was typed in correctly
    if ''.join(keys) != text[57:-2]:
        raise ValueError('There were typos. Please try again.')
        #return('There were typos. Please try again.')
    else:
        #contruct twograms from data
        twograms_data=[]
        for i in range(len(keys)-1):
            twograms_data.append((keys[i], keys[i+1]))
            
        #get time difference for corresponding twograms in data
        timediff=[]
        for i in range(len(times)-1):
            timediff.append(times[i+1]-times[i])
        
        #store data in dict
        data = {twograms_data[i]:timediff[i] for i in range(len(timediff))}
        return(data)
    
#function that initializes all possible twograms (in any alphabet) to zero and returns them in dict
def initialize_twograms(alphabet):
    twograms = []
    for l1 in alphabet:
        for l2 in alphabet:
            twograms.append((l1,l2))
    d = {twograms[i]:0 for i in range(len(twograms))}
    return(d)

def collect_data(text=text_train):
    user = initialize_twograms(alphabet)
    data = get_user_data(text)
    for key in set(data.keys()).intersection(set(user.keys())):
        user[key] = data[key]        
    return(user)

#function that calculates the mean squared error between values of dicts
def msq(d1, d2):    
    arr1 = np.array(list(d1.values()))
    arr2 = np.array(list(d2.values()))
    err = (arr1 - arr2)**2
    return np.mean(err) 

def test_program(users, text=text_test):  
    user_unknown = collect_data(text)
    #construct a matrix with the similarity scores
    scores = [msq(users[i], user_unknown) for i in range(len(users))]
    print('Mean Squared Error ={}\nThe unknown user is {}.'.format(scores, np.argmin(scores)+1 ) )
    return

In [12]:
#the alphabet with 27 letters that we will use
alphabet = list(map(chr, range(97, 123)))+ [' ']

In [20]:
#collect data 
user1 = collect_data('This is a test')

In [21]:
user2 = collect_data('This is a test')

In [24]:
#test program
test_program([user1, user2], 'The moment of truth')

Mean Squared Error =[0.013305606019846565, 0.015031558340377282]
The unknown user is 1.
